# Hubble Distance Modulus — Waveframe V4.0

This notebook computes the **luminosity distance** $D_L(z)$ and the **distance modulus** $\mu(z)$ for the Waveframe V4.0 H(z) model, and compares the results to ΛCDM predictions. The goal is to evaluate whether the model matches supernova data and remains consistent with observational constraints.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

# Constants
c = 299792.458  # speed of light in km/s
H0_LCDM = 70.0  # km/s/Mpc

# Example ΛCDM parameters
Omega_m = 0.3
Omega_L = 0.7

> **Automated patch:** Added figure-output utilities. Use `savefig('distance_modulus.png')` after plotting to save into `Figures/auto/`.

In [ ]:
# ---- Figure output setup (auto-creates Figures directories) ----
try:
    plt  # ensure matplotlib is available as plt
except NameError:
    import matplotlib.pyplot as plt

from pathlib import Path
from datetime import datetime

REPO_ROOT = Path(".").resolve()
FIG_DIR = REPO_ROOT / "Figures"
AUTO_DIR = FIG_DIR / "auto"
AUTO_DIR.mkdir(parents=True, exist_ok=True)

def savefig(name: str, fig=None, subdir: str = "auto", dpi=300):
    """Save current (or provided) figure into Figures/<subdir>/ with sane defaults."""
    target_dir = FIG_DIR / subdir if subdir else FIG_DIR
    target_dir.mkdir(parents=True, exist_ok=True)
    fig = fig or plt.gcf()
    out = target_dir / name
    fig.savefig(out, dpi=dpi, bbox_inches="tight")
    print(f"[saved] {out}")
    return out


**Tip:** After plotting μ(z) or D_L(z), call `savefig('distance_modulus.png')` or `savefig('luminosity_distance.png')` to persist the figure.

In [ ]:
def H_LCDM(z, H0=H0_LCDM, Om=Omega_m, OL=Omega_L):
    return H0 * np.sqrt(Om * (1+z)**3 + OL)

# Example Waveframe α(t) constant model (replace with full α(t) form as needed)
alpha_const = 0.5

def H_waveframe(z, H0=H0_LCDM, alpha=alpha_const):
    # Simple phenomenological deviation from LCDM
    return H_LCDM(z) * (1 + alpha * z / (1+z))

In [ ]:
def luminosity_distance(z, H_func):
    integral, _ = quad(lambda zp: 1.0 / H_func(zp), 0, z)
    return (c * integral)

def distance_modulus(z, H_func):
    DL = luminosity_distance(z, H_func)
    return 5 * np.log10(DL * 1e6 / 10)  # Mpc to pc, then to mag

In [ ]:
z_vals = np.linspace(0, 2, 200)
mu_LCDM = [distance_modulus(z, H_LCDM) for z in z_vals]
mu_WF = [distance_modulus(z, H_waveframe) for z in z_vals]

plt.figure(figsize=(8,5))
plt.plot(z_vals, mu_LCDM, label='ΛCDM', lw=2)
plt.plot(z_vals, mu_WF, label='Waveframe V4.0', lw=2, ls='--')
plt.xlabel('Redshift z')
plt.ylabel('Distance Modulus μ(z)')
plt.legend()
plt.title('Distance Modulus Comparison')
plt.grid(True)
plt.show()